# Q1
先用51的包，确认这些SA的函数定义没问题

In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [10]:
def parse_tsp(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
        cities = []
        for line in lines:
            parts = line.split()
            if len(parts) == 3 and parts[0].isdigit():
                cities.append([float(parts[1]), float(parts[2])])
        return np.array(cities)  # 使用NumPy数组

# 创建距离矩阵
def create_distance_matrix(cities):
    num_cities = len(cities)
    distance_matrix = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        distance_matrix[i] = np.sqrt(np.sum((cities - cities[i])**2, axis=1))
    return distance_matrix

# 使用距离矩阵计算路径长度
def path_length(distance_matrix, tour):
    return np.sum(distance_matrix[tour, np.roll(tour, -1)])

# 解析 .opt.tour 文件，以获取最优路径
def parse_opt_tour(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
        opt_tour = []
        for line in lines:
            parts = line.split()
            if parts[0].isdigit():
                opt_tour.append(int(parts[0]) - 1)
        return np.array(opt_tour)  # 使用NumPy数组

# 2-opt 算法的交换部分也可以使用NumPy进行优化，但这需要对算法的具体实现进行更深入的修改。


In [11]:
# 解析 eli51.tsp 和 eli51.opt.tour 文件
cities = parse_tsp('eil51.tsp.txt')
distance_matrix = create_distance_matrix(cities)
opt_tour = parse_opt_tour('eil51.opt.tour.txt')
opt_length = path_length(distance_matrix, opt_tour)




In [12]:
# 2-opt 算法
def two_opt_swap(tour, i, j):
    new_tour = list(tour[:i])
    new_tour.extend(reversed(tour[i:j + 1]))
    new_tour.extend(tour[j + 1:])
    return np.array(new_tour)

# 移动单个城市的算法
def move_city(tour, city_index, new_position):
    city = tour[city_index]
    new_tour = np.delete(tour, city_index)  # 删除原位置的城市
    new_tour = np.insert(new_tour, new_position, city)  # 在新位置插入城市
    return new_tour

In [13]:
# 不同的冷却计划
def linear_cooling(current_temp, alpha, iteration):
    return current_temp - alpha

def exponential_cooling(current_temp, alpha, iteration):
    return current_temp * alpha

def logarithmic_cooling(current_temp, alpha, iteration):
    return current_temp / (1 + alpha * np.log(1 + iteration))

In [14]:
# 2-opt策略的模拟退火算法
def simulated_annealing_2_opt(distance_matrix, initial_temp, final_temp, alpha, cooling_schedule):
    current_temp = initial_temp
    current_tour = np.random.permutation(len(distance_matrix))
    current_length = path_length(distance_matrix, current_tour)
    iteration = 1
    
    while current_temp > final_temp:
        i, j = np.sort(np.random.choice(len(cities), 2, replace=False))
        new_tour = two_opt_swap(current_tour, i, j)
        new_length = path_length(distance_matrix, new_tour)
        
        if new_length < current_length or np.random.rand() < np.exp((current_length - new_length) / current_temp):
            current_tour = new_tour
            current_length = new_length
            
        current_temp = cooling_schedule(current_temp, alpha, iteration)
        iteration += 1
        # current_temp *= alpha #默认使用exponential
    
    return current_tour, current_length

# 移动城市策略的模拟退火算法
def simulated_annealing_move_city(distance_matrix, initial_temp, final_temp, alpha, cooling_schedule):
    current_temp = initial_temp
    current_tour = np.random.permutation(len(distance_matrix))
    current_length = path_length(distance_matrix, current_tour)
    iteration = 1
    
    while current_temp > final_temp:
        city_index = np.random.randint(len(cities))
        new_position = np.random.randint(len(cities))
        new_tour = move_city(current_tour, city_index, new_position)
        new_length = path_length(distance_matrix, new_tour)
        
        if new_length < current_length or np.random.rand() < np.exp((current_length - new_length) / current_temp):
            current_tour = new_tour
            current_length = new_length
            
        current_temp = cooling_schedule(current_temp, alpha, iteration)
        iteration += 1
        # current_temp *= alpha #默认使用exponential

    return current_tour, current_length



In [15]:
# 运行模拟退火算法多次
def multiple_runs_simulated_annealing(distance_matrix, runs, initial_temp, final_temp, alpha, strategy, cooling_schedule):
    best_tour = None
    best_length = float('inf')
    
    for _ in range(runs):
        if strategy == '2-opt':
            tour, length = simulated_annealing_2_opt(distance_matrix, initial_temp, final_temp, alpha, cooling_schedule)
        elif strategy == 'move_city':
            tour, length = simulated_annealing_move_city(distance_matrix, initial_temp, final_temp, alpha, cooling_schedule)
        
        if length < best_length:
            best_length = length
            best_tour = tour
    
    return best_tour, best_length

# 设置参数
runs = 100  # 运行次数
initial_temp = 15000  # 初始温度
final_temp = 0.001    # 最终温度
alpha = 0.999         # 冷却率

# 运行模拟退火算法多次，使用2-opt
best_tour_2opt, best_length_2opt = multiple_runs_simulated_annealing(distance_matrix, runs, initial_temp, final_temp, alpha, '2-opt', exponential_cooling)
# 运行模拟退火算法多次，使用移动城市策略
best_tour_move_city, best_length_move_city = multiple_runs_simulated_annealing(distance_matrix, runs, initial_temp, final_temp, alpha, 'move_city', exponential_cooling)

print("使用2-opt找到的最佳路径长度:", best_length_2opt)
print("使用移动城市策略找到的最佳路径长度:", best_length_move_city)
print("最优路径长度:", opt_length)

使用2-opt找到的最佳路径长度: 438.27221418121366
使用移动城市策略找到的最佳路径长度: 453.6134683169201
最优路径长度: 429.983311983384


# 跑一下280的包

In [23]:
# 导入280数据
cities = parse_tsp('a280.tsp.txt')
distance_matrix = create_distance_matrix(cities)
opt_tour = parse_opt_tour('a280.opt.tour.txt')
opt_length280 = path_length(distance_matrix, opt_tour)


# 设置参数
runs = 200  # 运行次数
initial_temp = 15000  # 初始温度
final_temp = 0.001    # 最终温度
alpha = 0.997         # 冷却率

# 运行模拟退火算法多次，使用2-opt
best_tour_2opt_280, best_length_2opt_280 = multiple_runs_simulated_annealing(distance_matrix, runs, initial_temp, final_temp, alpha, '2-opt', exponential_cooling)

# 运行模拟退火算法多次，使用移动城市策略
best_tour_move_city_280, best_length_move_city_280 = multiple_runs_simulated_annealing(distance_matrix, runs, initial_temp, final_temp, alpha, 'move_city', exponential_cooling)

print("使用2-opt找到的最佳路径长度:", best_length_2opt_280)
print("使用移动城市策略找到的最佳路径长度:", best_length_move_city_280)
print("最优路径长度:", opt_length280)

使用2-opt找到的最佳路径长度: 9477.890027246616
使用移动城市策略找到的最佳路径长度: 11588.995858856464
最优路径长度: 2586.7696475631597


# Q2

针对280文件包进行，接下来的分析都围绕这个包展开

In [20]:
# 导入280数据
cities280 = parse_tsp('a280.tsp.txt')
opt_tour280 = parse_opt_tour('a280.opt.tour.txt')
opt_length280 = calculate_opt_length(cities, opt_tour)

NameError: name 'distance' is not defined

In [ ]:
# 设置参数
runs = 20  # 每种策略和冷却计划的运行次数
initial_temp = 12000  # 初始温度
final_temp = 1        # 最终温度
alpha = 0.995          # 冷却率

# 运行模拟退火算法并收集结果
results = {}
for strategy, func in [('2-opt', simulated_annealing_2_opt), ('move_city', simulated_annealing_move_city)]:
    for cooling_schedule in [linear_cooling, exponential_cooling, logarithmic_cooling]:
        total_length = 0
        for _ in range(runs):
            tour, length = func(cities280, initial_temp, final_temp, alpha, cooling_schedule)
            total_length += length
        average_length = total_length / runs
        results[(strategy, cooling_schedule.__name__)] = average_length

# 打印结果
for key, value in results.items():
    print(f"策略: {key[0]}, 冷却计划: {key[1]}, 平均路径长度: {value}")


In [ ]:

# 多次运行模拟退火算法
def multiple_runs_simulated_annealing(cities, runs, initial_temp, final_temp, alpha, cooling_schedule):
    best_length = float('inf')
    
    for _ in range(runs):
        length = simulated_annealing_2_opt(cities, initial_temp, final_temp, alpha, cooling_schedule)
        if length < best_length:
            best_length = length
    
    return best_length

# 解析 TSP 和 OPT 文件
cities = parse_tsp('a280.tsp.txt')  # 替换为 a280.tsp 文件的实际路径
opt_tour = parse_opt_tour('a280.opt.tour.txt')  # 替换为 a280.opt.tour 文件的实际路径
opt_length = calculate_opt_length(cities, opt_tour)

# 设置参数并运行模拟退火算法多次
runs = 3  # 运行次数
initial_temp = 10000  # 初始温度
final_temp = 1        # 最终温度
alpha = 0.997         # 冷却率

# 运行模拟退火算法并比较不同冷却计划
cooling_strategies = {
    "Linear Cooling": lambda temp, alpha, _: linear_cooling(current_temp, alpha, iteration),
    "Exponential Cooling": lambda temp, alpha, _: exponential_cooling(current_temp, alpha, iteration),
    "Logarithmic Cooling": logarithmic_cooling
}

results = {}
for name, strategy in cooling_strategies.items():
    best_length = multiple_runs_simulated_annealing(cities, runs, initial_temp, final_temp, alpha, strategy)
    results[name] = best_length
    print(f"{name}: {best_length}")

# 可视化结果
plt.figure(figsize=(10, 6))
plt.bar(results.keys(), results.values(), color=['blue', 'green', 'red'])
plt.plot(['Optimal Length'] * 2, [opt_length, opt_length], color='black', linestyle='--')
plt.text(-0.5, opt_length, f'Optimal Length: {opt_length}', verticalalignment='bottom')
plt.xlabel('Cooling Schedule')
plt.ylabel('Path Length')
plt.title('Comparison of Different Cooling Schedules with Optimal Path')
plt.show()

In [ ]:
import numpy as np


# 之前的函数定义（parse_tsp, distance, path_length, two_opt_swap, parse_opt_tour, calculate_opt_length）

def two_opt_swap(tour, i, j):
    new_tour = np.concatenate([tour[:i], tour[i:j + 1][::-1], tour[j + 1:]])
    return new_tour

def simulated_annealing(cities, initial_temp, final_temp, alpha, cooling_schedule):
    current_temp = initial_temp
    best_tour = np.random.permutation(len(cities))
    best_length = path_length(cities, best_tour)
    iteration = 1
    lengths = []  # 用于记录每次迭代的最佳路径长度

    while current_temp > final_temp:
        for _ in range(len(cities)):
            i, j = np.sort(np.random.choice(len(cities), 2, replace=False))
            new_tour = two_opt_swap(best_tour, i, j)
            new_length = path_length(cities, new_tour)

            if new_length < best_length or np.random.rand() < np.exp((best_length - new_length) / current_temp):
                best_tour = new_tour
                best_length = new_length

        lengths.append(best_length)  # 记录当前最佳长度
        current_temp = cooling_schedule(current_temp, alpha, iteration)
        iteration += 1

    return best_length, lengths

# 不同的冷却计划定义（linear_cooling, exponential_cooling, logarithmic_cooling）

def multiple_runs_simulated_annealing(cities, runs, initial_temp, final_temp, alpha, cooling_schedule):
    best_lengths = []
    
    for _ in range(runs):
        _, lengths = simulated_annealing(cities, initial_temp, final_temp, alpha, cooling_schedule)
        best_lengths.append(lengths)
    
    return best_lengths



# 设置参数
runs = 5  # 运行次数减少以减少计算时间
initial_temp = 10000
final_temp = 0.01
alpha = 0.997

# 绘制收敛速度图
plt.figure(figsize=(15, 5))
for name, strategy in cooling_strategies.items():
    lengths = multiple_runs_simulated_annealing(cities, runs, initial_temp, final_temp, alpha, strategy)
    avg_lengths = np.mean(lengths, axis=0)
    plt.plot(avg_lengths, label=name)

plt.axhline(y=opt_length, color='r', linestyle='--', label='Optimal Length')
plt.xlabel('Iteration')
plt.ylabel('Path Length')
plt.title('Convergence Speed of Different Cooling Schedules')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import scipy.stats as stats

# 之前定义的函数，比如 simulated_annealing_2_opt, simulated_annealing_move_city, path_length, distance, parse_tsp 等

# 数据收集和准备
def collect_data_for_ttest(cities, runs, initial_temp, final_temp, alpha, cooling_schedule):
    lengths = []
    for _ in range(runs):
        _, length = simulated_annealing_2_opt(cities, initial_temp, final_temp, alpha, cooling_schedule)
        lengths.append(length)
    return lengths

# 参数设置
runs = 30  # 每种冷却计划运行次数
initial_temp = 10000
final_temp = 1
alpha = 0.995

# 收集不同冷却计划下的数据
lengths_linear = collect_data_for_ttest(cities, runs, initial_temp, final_temp, alpha, linear_cooling)
lengths_exponential = collect_data_for_ttest(cities, runs, initial_temp, final_temp, alpha, exponential_cooling)
lengths_logarithmic = collect_data_for_ttest(cities, runs, initial_temp, final_temp, alpha, logarithmic_cooling)

# 进行 t 检验
t_test_results = {}
t_test_results['linear vs exponential'] = stats.ttest_ind(lengths_linear, lengths_exponential)
t_test_results['linear vs logarithmic'] = stats.ttest_ind(lengths_linear, lengths_logarithmic)
t_test_results['exponential vs logarithmic'] = stats.ttest_ind(lengths_exponential, lengths_logarithmic)

# 打印 t 检验结果
for test, result in t_test_results.items():
    print(f"{test}: statistic={result.statistic}, p-value={result.pvalue}")


# Q3


In [ ]:
def exponential_cooling(current_temp, alpha, iteration):
    return current_temp * alpha

def simulated_annealing(cities, initial_temp, final_temp, alpha, cooling_schedule, markov_chain_length):
    current_temp = initial_temp
    best_tour = np.random.permutation(len(cities))
    best_length = path_length(cities, best_tour)
    iteration = 1  # 初始化迭代变量
    lengths = [best_length]

    while current_temp > final_temp:
        for _ in range(markov_chain_length):
            i, j = np.sort(np.random.choice(len(cities), 2, replace=False))
            new_tour = two_opt_swap(best_tour, i, j)
            new_length = path_length(cities, new_tour)

            if new_length < best_length or np.random.rand() < np.exp((best_length - new_length) / current_temp):
                best_tour = new_tour
                best_length = new_length
        
        lengths.append(best_length)  # 在每次温度变化后记录路径长度
        current_temp = cooling_schedule(current_temp, alpha, iteration)  # 使用当前迭代次数更新温度
        iteration += 1  # 增加迭代次数

    return best_length, lengths


markov_lengths = [1, 10, 50, 100, 200]  # 可根据需要调整这些值
results = {}

for length in markov_lengths:
    _, lengths = simulated_annealing(cities, initial_temp, final_temp, alpha, exponential_cooling, length)
    results[length] = lengths
    
# 修改绘图代码部分
plt.figure(figsize=(15, 5))
for length in markov_lengths:
    _, lengths = simulated_annealing(cities, initial_temp, final_temp, alpha, exponential_cooling, length)
    plt.plot(range(len(lengths)), lengths, label=f'Markov Chain Length {length}')  # 使用正确的迭代次数范围

plt.axhline(y=opt_length, color='r', linestyle='--', label='Optimal Length')
plt.xlabel('Iteration')
plt.ylabel('Path Length')
plt.title('Effects of Markov Chain Length on Convergence')
plt.legend()
plt.show()